# LSTM Model With Attention



In [1]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

Using TensorFlow backend.


In [2]:
# Keras
from keras.preprocessing import text, sequence
import pandas as pd
import helpers as hlp


def make_df(train_path, test_path, max_features, maxlen, list_classes):
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)
    train = train.sample(frac=1)

    list_sentences_train = train["comment_text"].fillna("unknown").values
    y = train[list_classes].values
    list_sentences_test = test["comment_text"].fillna("unknown").values
    
    #list_sentences_train = hlp.clean(list_sentences_train)
    #list_sentences_test = hlp.clean(list_sentences_test)

    tokenizer = text.Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(list_sentences_train))
    list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
    list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
    X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
    X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)
    
    word_index = tokenizer.word_index

    return X_t, X_te, y, word_index

In [3]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout


def BidLstm(maxlen, max_features, embed_size, embedding_matrix):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix],
                  trainable=False)(inp)
    x = Bidirectional(LSTM(512, return_sequences=True, dropout=0.2,
                           recurrent_dropout=0.2))(x)
    x = Attention(maxlen)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.4)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)

    return model

In [4]:
import numpy as np


def make_glovevec(glovepath, max_features, embed_size, word_index, veclen=300):
    embeddings_index = {}
    f = open(glovepath)
    for line in f:
        values = line.split()
        word = ' '.join(values[:-300])
        coefs = np.asarray(values[-300:], dtype='float32')
        embeddings_index[word] = coefs.reshape(-1)
    f.close()

    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    print(embedding_matrix)
    return embedding_matrix

In [5]:
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint
np.random.seed(7)


if __name__ == "__main__":
    max_features = 100000
    maxlen = 150
    embed_size = 300
    list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult",
                    "identity_hate"]

    xtr, xte, y, word_index = make_df("./input/train.csv",
                                      "./input/test.csv",
                                      max_features, maxlen, list_classes)
    embedding_vector = make_glovevec("./input/glove.840B.300d.txt",
                                     max_features, embed_size, word_index)

    model = BidLstm(maxlen, max_features, embed_size, embedding_vector)
    model.compile(loss='binary_crossentropy', optimizer='adam',
                  metrics=['accuracy'])
    file_path = ".model.hdf5"
    ckpt = ModelCheckpoint(file_path, monitor='val_loss', verbose=1,
                           save_best_only=True, mode='min')
    early = EarlyStopping(monitor="val_loss", mode="min", patience=3)
    model.fit(xtr, y, batch_size=512, epochs=20, validation_split=0.1, callbacks=[ckpt, early])
    #model.fit(xtr, y, batch_size=256, epochs=1, validation_split=0.1)
    model.load_weights(file_path)
    print("Predicting with model...")
    y_test = model.predict(xte)
    sample_submission = pd.read_csv("./input/sample_submission.csv")
    sample_submission[list_classes] = y_test
    print("Saving to submission file...")
    sample_submission.to_csv("lstm_sub.csv", index=False)

    

[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.27204001 -0.06203    -0.1884     ...,  0.13015001 -0.18317001  0.1323    ]
 [ 0.31924     0.06316    -0.27858001 ...,  0.082745    0.097801
   0.25044999]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [-0.37496999 -0.37419999  0.067547   ..., -0.026452   -0.23654    -0.037388  ]]
Train on 143613 samples, validate on 15958 samples
Epoch 1/20
143360/143613 [============================>.] - ETA: 0s - loss: 0.0756 - acc: 0.9757
Epoch 00001: val_loss improved from inf to 0.04923, saving model to .model.hdf5
143613/143613 [==============================] - 164s 1ms/step - loss: 0.0755 - acc: 0.9758 - val_loss: 0.0492 - val_acc: 0.9821
Epoch 2/20
143360/143613 [============================>.] - ETA: 0s - loss: 0.0487 - acc: 0.9819
Epoch 00002: val_loss improved from 0.04923 to 0.04534, sa

In [8]:
model.load_weights(file_path)
y_test = model.predict(xte)
sample_submission = pd.read_csv("./input/sample_submission.csv")
sample_submission[list_classes] = y_test
sample_submission.to_csv("submissions/bidlstm_with_attention.csv", index=False)

NameError: name 'model' is not defined

In [58]:
model.save('my_model.h5')